# Find model/scenario sets with complete data
We wish to find the most complete set of variables possible. We will restrict ourselves to the first order (i.e. CO2 but not subtypes) and find the subset of scenarios which have all of them. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyam
import silicone

import silicone.multiple_infillers as mi
import silicone.database_crunchers as cr
from silicone.utils import (
    download_or_load_sr15,
)

In c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\iiasa-climate\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\iiasa-climate\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\iiasa-climate\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


<IPython.core.display.Javascript object>

pyam - INFO: Running in a notebook, setting `pyam` logging level to `logging.INFO` and adding stderr handler


In [2]:
sr15_file = "./../input/complete_sr15.csv"
all_models = [
    'AIM/CGE 2.0', 'AIM/CGE 2.1', 'C-ROADS-5.005', 'GCAM 4.2', 'GENeSYS-MOD 1.0', 
    'IEA Energy Technology Perspective Model 2017', 'IEA World Energy Model 2017', 
    'IMAGE 3.0.1', 'IMAGE 3.0.2', 'MERGE-ETL 6.0', 'MESSAGE V.3', 'MESSAGE-GLOBIOM 1.0', 
    'MESSAGEix-GLOBIOM 1.0', 'POLES ADVANCE', 'POLES CD-LINKS', 'POLES EMF33', 'REMIND 1.5', 
    'REMIND 1.7', 'REMIND-MAgPIE 1.5', 'REMIND-MAgPIE 1.7-3.0', 
    'Shell World Energy Model 2018', 'WITCH-GLOBIOM 3.1', 'WITCH-GLOBIOM 4.2', 'WITCH-GLOBIOM 4.4'
]

df = download_or_load_sr15(sr15_file, all_models).filter(region="World") 
df.head()

pyam.core - INFO: Reading file `./../input/complete_sr15.csv`


,model,scenario,region,variable,unit,year,value
0,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,World,Emissions|BC,Mt BC/yr,2005,8.3367
1,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,World,Emissions|BC,Mt BC/yr,2010,7.6408
2,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,World,Emissions|BC,Mt BC/yr,2015,7.2724
3,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,World,Emissions|BC,Mt BC/yr,2020,7.0752
4,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,World,Emissions|BC,Mt BC/yr,2025,5.8486


In [3]:
vars = df.filter(level=1).variables()
var_len = {}
ms = ["model", "scenario"]
accepted_mscen = df.filter(variable="*PFC").data[["model", "scenario"]].drop_duplicates()
for var in vars:
    var_len[var] = len(df.filter(variable=var).timeseries())
    accepted_mscen = accepted_mscen.merge(
        df.filter(variable=var).timeseries(), 
        on=ms, 
        how='left', 
        indicator=True,
    )
    accepted_mscen = accepted_mscen.loc[accepted_mscen._merge=="both"][ms]

In [4]:
accepted = df.data.merge(accepted_mscen, on=ms, how="left", indicator=True)
accepted = accepted[accepted._merge=="both"]

In [5]:
del accepted["_merge"]
accepted = pyam.IamDataFrame(accepted)
print(accepted.head())

         model                           scenario region      variable  \
0  AIM/CGE 2.1  TERL_15D_LowCarbonTransportPolicy  World  Emissions|BC   
1  AIM/CGE 2.1  TERL_15D_LowCarbonTransportPolicy  World  Emissions|BC   
2  AIM/CGE 2.1  TERL_15D_LowCarbonTransportPolicy  World  Emissions|BC   
3  AIM/CGE 2.1  TERL_15D_LowCarbonTransportPolicy  World  Emissions|BC   
4  AIM/CGE 2.1  TERL_15D_LowCarbonTransportPolicy  World  Emissions|BC   

       unit  year   value  
0  Mt BC/yr  2005  9.1717  
1  Mt BC/yr  2010  9.8494  
2  Mt BC/yr  2015  9.9234  
3  Mt BC/yr  2020  9.7993  
4  Mt BC/yr  2025  9.1199  


In [ ]:
accepted.filter(variable="*CO2").plot()
print(len())

## Save the results to files

In [12]:
accepted.to_csv("../output/allvar_sr15_mscen.csv")
accepted.filter(variable="*CO2*").to_csv("../output/allvar_co2_only_sr15_mscen.csv")
accepted.filter(variable=["*CO2*", "*CH4*"]).to_csv("../output/allvar_co2ch4_only_sr15_mscen.csv")